In [1]:
import findspark
findspark.init()

import pyspark

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [3]:
spark = SparkSession.builder.appName('pySpark Visualisation').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/23 11:28:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
print("ok")

ok


In [6]:
import os

In [7]:
ls

EI_DJTA_20160128_Trades.csv       newData_timeSeries.ipynb
Iris.csv                          pySpark_Iris_Visualisation.ipynb


In [8]:
dataDF = spark.read.csv("Iris.csv",header= True, inferSchema= True)

In [9]:
dataDF.show(truncate= False)

+---+-------------+------------+-------------+------------+-----------+
|Id |SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species    |
+---+-------------+------------+-------------+------------+-----------+
|1  |5.1          |3.5         |1.4          |0.2         |Iris-setosa|
|2  |4.9          |3.0         |1.4          |0.2         |Iris-setosa|
|3  |4.7          |3.2         |1.3          |0.2         |Iris-setosa|
|4  |4.6          |3.1         |1.5          |0.2         |Iris-setosa|
|5  |5.0          |3.6         |1.4          |0.2         |Iris-setosa|
|6  |5.4          |3.9         |1.7          |0.4         |Iris-setosa|
|7  |4.6          |3.4         |1.4          |0.3         |Iris-setosa|
|8  |5.0          |3.4         |1.5          |0.2         |Iris-setosa|
|9  |4.4          |2.9         |1.4          |0.2         |Iris-setosa|
|10 |4.9          |3.1         |1.5          |0.1         |Iris-setosa|
|11 |5.4          |3.7         |1.5          |0.2         |Iris-

In [13]:
dataDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



In [16]:
# shape of dataFrame - 
print(dataDF.count(),len(dataDF.columns))

150 6


In [19]:
# describle equivalent - 
dataDF.summary().show()

+-------+------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|                Id|     SepalLengthCm|       SepalWidthCm|     PetalLengthCm|      PetalWidthCm|       Species|
+-------+------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|               150|                150|               150|               150|           150|
|   mean|              75.5| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|          null|
| stddev|43.445367992456916|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|          null|
|    min|                 1|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    25%|                38|               5.1|                2.8|               1.6|               0.3|          null|
|    50%|                75|    

In [21]:
# info equivalent
dataDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



In [22]:
dataDF.isNull()

AttributeError: 'DataFrame' object has no attribute 'isNull'

In [27]:
#  to check for isNull() in all columns
tempDF = dataDF.select([count(when(col(c).contains('None') \
                                   | col(c).contains('NULL') \
                                   | (col(c) == '') \
                                   | col(c).isNull() \
                                   | isnan(c),c )).alias( c) for c in dataDF.columns])
tempDF.show()

+---+-------------+------------+-------------+------------+-------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species|
+---+-------------+------------+-------------+------------+-------+
|  0|            0|           0|            0|           0|      0|
+---+-------------+------------+-------------+------------+-------+



In [30]:
dataDF.select([col(c).distinct().count().alias(c)  for c in dataDF.columns ] ).show

TypeError: 'Column' object is not callable

In [36]:
dataDF.select([countDistinct(c) for c in dataDF.columns]).show()

+------------------+-----------------------------+----------------------------+-----------------------------+----------------------------+-----------------------+
|count(DISTINCT Id)|count(DISTINCT SepalLengthCm)|count(DISTINCT SepalWidthCm)|count(DISTINCT PetalLengthCm)|count(DISTINCT PetalWidthCm)|count(DISTINCT Species)|
+------------------+-----------------------------+----------------------------+-----------------------------+----------------------------+-----------------------+
|               150|                           35|                          23|                           43|                          22|                      3|
+------------------+-----------------------------+----------------------------+-----------------------------+----------------------------+-----------------------+



In [10]:
unique_names = []
for x in dataDF.select("Species").distinct().collect():
    unique_names.append(x[0])

In [11]:
print(unique_names)

['Iris-virginica', 'Iris-setosa', 'Iris-versicolor']


In [12]:
tables = [dataDF.where(dataDF.Species == x) for x in unique_names]

In [14]:
for table in tables:
    table.show()

+---+-------------+------------+-------------+------------+--------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|       Species|
+---+-------------+------------+-------------+------------+--------------+
|101|          6.3|         3.3|          6.0|         2.5|Iris-virginica|
|102|          5.8|         2.7|          5.1|         1.9|Iris-virginica|
|103|          7.1|         3.0|          5.9|         2.1|Iris-virginica|
|104|          6.3|         2.9|          5.6|         1.8|Iris-virginica|
|105|          6.5|         3.0|          5.8|         2.2|Iris-virginica|
|106|          7.6|         3.0|          6.6|         2.1|Iris-virginica|
|107|          4.9|         2.5|          4.5|         1.7|Iris-virginica|
|108|          7.3|         2.9|          6.3|         1.8|Iris-virginica|
|109|          6.7|         2.5|          5.8|         1.8|Iris-virginica|
|110|          7.2|         3.6|          6.1|         2.5|Iris-virginica|
|111|          6.5|      

In [13]:
for df in tables:
    print(df.stat.corr('SepalLengthCm','SepalWidthCm'),df.stat.corr('PetalLengthCm','PetalWidthCm'),df.Species)  

0.45722781639411325 0.3221082159003181 Column<'Species'>
0.7467803732639264 0.30630821115803564 Column<'Species'>
0.5259107172828247 0.786668088522817 Column<'Species'>


### New Work


In [15]:
data = [("James","M",60000),("Michael","M",70000),
        ("Robert",None,400000),("Maria","F",500000),
        ("Jen","",None)]

columns = ["name","gender","salary"]
df = spark.createDataFrame(data = data, schema = columns)
df.show()

+-------+------+------+
|   name|gender|salary|
+-------+------+------+
|  James|     M| 60000|
|Michael|     M| 70000|
| Robert|  null|400000|
|  Maria|     F|500000|
|    Jen|      |  null|
+-------+------+------+



In [19]:
df2 = df.withColumn("new_gender", F.when(df.gender == "M", "Male").when(df.gender == "F", "Female").when(df.gender.isNull(), "").otherwise(df.gender))
df2.show()
                    

+-------+------+------+----------+
|   name|gender|salary|new_gender|
+-------+------+------+----------+
|  James|     M| 60000|      Male|
|Michael|     M| 70000|      Male|
| Robert|  null|400000|          |
|  Maria|     F|500000|    Female|
|    Jen|      |  null|          |
+-------+------+------+----------+



In [26]:
df3 = df.withColumn("new_gender", F.expr("CASE WHEN gender = 'M' THEN 'Male' " + 
               "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
               "ELSE gender END"))
df3.show(truncate=False)

+-------+------+------+----------+
|name   |gender|salary|new_gender|
+-------+------+------+----------+
|James  |M     |60000 |Male      |
|Michael|M     |70000 |Male      |
|Robert |null  |400000|          |
|Maria  |F     |500000|Female    |
|Jen    |      |null  |          |
+-------+------+------+----------+



In [37]:
#  when(condition, do x).otherwise(y)

data = [("111",50000),("222",60000),("333",40000)]
columns= ["EmpId","Salary"]
df = spark.createDataFrame(data = data, schema = columns)
df2 = df.select(col("EmpId"),col("Salary"),lit("1").alias("lit_value1"))
df2.show(truncate=False)



+-----+------+----------+
|EmpId|Salary|lit_value1|
+-----+------+----------+
|111  |50000 |1         |
|222  |60000 |1         |
|333  |40000 |1         |
+-----+------+----------+



In [41]:
# df3 = df2.withColumn("lit_value2", when(col("Salary") >= 40000 & col("Salary")  < 50000, lit("100")).otherwise(lit("200")))
df3 = df2.withColumn("lit_value2", when((col("Salary") >=40000) & (col("Salary") <= 50000),lit("100")).otherwise(lit("200")))

In [42]:
df3.show()

+-----+------+----------+----------+
|EmpId|Salary|lit_value1|lit_value2|
+-----+------+----------+----------+
|  111| 50000|         1|       100|
|  222| 60000|         1|       200|
|  333| 40000|         1|       100|
+-----+------+----------+----------+



In [43]:
#  df.select(split(df.name,',').alias("nameArray")).drop("name")
# string to array_of_string -> split *using delimiter*
# array to string -> concat_ws
#  Note -> use 'col()' as it takes in col argument in second
columns = ["name","languagesAtSchool","currentState"]
data = [("James,,Smith",["Java","Scala","C++"],"CA"), \
    ("Michael,Rose,",["Spark","Java","C++"],"NJ"), \
    ("Robert,,Williams",["CSharp","VB"],"NV")]

df = spark.createDataFrame(data=data,schema=columns)
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- languagesAtSchool: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- currentState: string (nullable = true)

+----------------+------------------+------------+
|name            |languagesAtSchool |currentState|
+----------------+------------------+------------+
|James,,Smith    |[Java, Scala, C++]|CA          |
|Michael,Rose,   |[Spark, Java, C++]|NJ          |
|Robert,,Williams|[CSharp, VB]      |NV          |
+----------------+------------------+------------+



In [44]:
df2 = df.withColumn("Column_Array", concat_ws(" ",col("languagesAtSchool")))

In [45]:
df2.show()

+----------------+------------------+------------+--------------+
|            name| languagesAtSchool|currentState|  Column_Array|
+----------------+------------------+------------+--------------+
|    James,,Smith|[Java, Scala, C++]|          CA|Java Scala C++|
|   Michael,Rose,|[Spark, Java, C++]|          NJ|Spark Java C++|
|Robert,,Williams|      [CSharp, VB]|          NV|     CSharp VB|
+----------------+------------------+------------+--------------+



In [46]:
#  regex replace ->. regex replace(name of column , string value to be replaced, string value to replace by )
#  there is overlay as well


In [49]:
from pyspark.sql.types import StringType

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)
df.show(truncate=False)














+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+



In [50]:
def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr 

In [51]:
""" Converting function to UDF """
convertUDF = udf(lambda z: convertCase(z))

In [52]:
df.select(col("Seqno"), \
    convertUDF(col("Name")).alias("Name") ) \
.show(truncate=False)

+-----+-------------+
|Seqno|Name         |
+-----+-------------+
|1    |John Jones   |
|2    |Tracey Smith |
|3    |Amy Sanders  |
+-----+-------------+



In [53]:
def upperCase(str):
    return str.upper()

In [54]:
upperCaseUDF = udf(lambda z:upperCase(z),StringType())    

df.withColumn("Cureated Name", upperCaseUDF(col("Name"))) \
.show(truncate=False)

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+



In [ ]:
""" Using UDF on SQL """
spark.udf.register("convertUDF", convertCase,StringType())
df.createOrReplaceTempView("NAME_TABLE")
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE") \
     .show(truncate=False)
     
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE " + \
          "where Name is not null and convertUDF(Name) like '%John%'") \
     .show(truncate=False)  
     
""" null check """

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")
    
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False)  